# <span style="color:purple">Script edit dot</span>

In [90]:
import pydot
from pydot import Edge
from pydot import Node
import re
global subgraphs, parents
subgraphs={}
parents={}

## Our goal
We want to change the following graph in order to have new nodes to visualize the flow between functions
<img src="graph.png" style="width:400px">

#### Import a graph from dot and return the pydot graph

In [91]:
def graph_from_dot(file):
    """
       Import a graph from dot and return the pydot graph
       
       :param file: path to the file to import
       :return: pydot graph
       :rtype: Dot
    """
    (graph, ) = pydot.graph_from_dot_file(str(file))
    return graph

In [92]:
G = graph_from_dot("../Tests/while_with_foo/test.c.011t.cfg.dot");

#### Extract subgraphs, edges, nodes

In [93]:
def extract_subgraphs(subgraph_list):
    global subgraphs, parents
    """
       Extract subgraphs recursively from a list of subgraphs, and the hierarchy of the subgraphs
       
       :param graph: graph or subgraph from which subgraphs are to be extracted
       :return: List of subgraphs
       :rtype: List
    """
    new_subgraphs = []
    for subgraph in subgraph_list:
        subgraph.set_name(subgraph.get_name().strip("\"").replace("cluster_", ""))
        subgraphs[subgraph.get_name()] = subgraph
        children_subgraphs = subgraph.get_subgraph_list() 
        new_subgraphs += children_subgraphs
        for child in children_subgraphs:
            parents[child.get_name().strip("\"").replace("cluster_", "")] = subgraph
    if(new_subgraphs == []):
        return
    else:
        return extract_subgraphs(new_subgraphs)

In [94]:
extract_subgraphs([G])
print(subgraphs)

{'test.c.011t.cfg': <pydot.Dot object at 0x7f36c11bba20>, 'foo': <pydot.Subgraph object at 0x7f36c1278b70>, 'main': <pydot.Subgraph object at 0x7f36c1647a20>, '1_1': <pydot.Subgraph object at 0x7f36c12126d8>}


In [95]:
def extract_nodes(graph):
    """
       Extract nodes from a graph or subgraph
       
       :param graph: graph or subgraph from which nodes are to be extracted
       :return: List of nodes
       :rtype: List
    """
    return graph.get_nodes()

In [96]:
nodes_main = extract_nodes(subgraphs['main'])
print(nodes_main)

[<pydot.Node object at 0x7f36c146f470>, <pydot.Node object at 0x7f36c146f438>, <pydot.Node object at 0x7f36c11f5ba8>, <pydot.Node object at 0x7f36c11d6240>, <pydot.Node object at 0x7f36c11e3898>]


In [97]:
def extract_edges(graph):
    """
       Extract edges from a graph or subgraph
       
       :param graph: graph or subgraph from which edges are to be extracted
       :return: List of edges
       :rtype: List
    """
    return graph.get_edges()

In [98]:
edges_main = extract_edges(subgraphs['main'])
print(edges_main)

[<pydot.Edge object at 0x7f36c165fe80>, <pydot.Edge object at 0x7f36c15e2518>, <pydot.Edge object at 0x7f36c1648eb8>, <pydot.Edge object at 0x7f36c1648fd0>, <pydot.Edge object at 0x7f36c164a9b0>, <pydot.Edge object at 0x7f36c1237cc0>, <pydot.Edge object at 0x7f36c1210358>, <pydot.Edge object at 0x7f36c121eef0>, <pydot.Edge object at 0x7f36c1206048>, <pydot.Edge object at 0x7f36c120c6a0>, <pydot.Edge object at 0x7f36c1211cf8>]


#### Add an edge, a node

In [99]:
def add_edge(graph, tail_node, head_node, label="[0%]", style="solid, bold", color="black"):
    """
       Add an edge to a graph or subgraph
       
       :param graph: graph or subgraph to which the edge is added
       :param tail_node: origin of the edge
       :param head_node: destination of the edge
    """
    edge = Edge(tail_node.get_name()+":s", head_node.get_name()+":n", label=label, style=style, color=color)
    graph.add_edge(edge)
    return

In [100]:
add_edge(subgraphs['main'], nodes_main[0], nodes_main[1], color="red")

In [101]:
def add_node(graph, node_name, label, shape='record', style='filled', fillcolor='lightgrey'):
    """
       Add a node to a graph or subgraph
       
       :param graph: graph or subgraph to which the node is added
       :param node_name: name of the node
       :param label: label of the node
       :param shape: shape of the node (default "record")
       :param style: style of the node (default "field")
       :param fillcolor: color of the node (default "lightgrey")
    """
    node = Node(name=node_name, shape=shape, style=style, fillcolor=fillcolor, label=label)
    graph.add_node(node)
    return node

In [102]:
add_node(G, 'node_yolo', 'yolo')

#### Analyse and create labels if a function call is found

In [103]:
def analyse_label(node):
    """
        Find if it exists, the first call to a function in a node
        
        :param node: the node examined
        :return: tuple containing the name of the function and its position in the label of the node
        :rtype: tuple
    """
    global subgraphs
    for match in re.finditer(r'\w+\\\s\(\w*\)', node.get_attributes()['label']):
        function_name = re.search(r'\w+', match.group())
        if function_name is not None:
            if function_name.group() in subgraphs:
                return (function_name.group(), match.group())
    return (None, None)

In [104]:
node = extract_nodes(subgraphs['1_1'])[0]
analyse = analyse_label(node)
print(analyse)

('foo', 'foo\\ (b)')


In [105]:
def create_labels(node, function_call):
    """
        Create the labels for the new node, and for the node cut in half.
        
        :param node: the node cut
        :param function_call: the string containing the function call used as a regex expression
        :return: tuple containing the new label for the node cut, the label for the new node, and the value of
                 the cut node's bb
        :rtype: tuple
    """
    label_node = node.get_attributes()["label"]
    
    match_function = re.search(r'\\l\\(.(?!\\l))*' + re.escape(function_call) + r';\\l\\', label_node, re.S)
    
    bb = int(re.search(r'\d+', re.search(r'<bb\\\ \d+\\>', label_node).group()).group())
    label_node1 = label_node[0:match_function.start()] + '\l\\\n|call\ ' + function_call + ';\l\\\n}"'
    
    match_variable = re.search(r'\w+\\ =\ ' + function_call, label_node)
    label_node2 = '"{ FREQ:0 |\<bb\ ' + str(bb+1) + '\>:\l\|'
    if match_variable is not None:
        variable = re.search(r'\w+', match_variable.group()).group()
        label_node2 += variable + '\ =\ '
    label_node2 += 'return\ ' + function_call + ';\\l\\\n' + label_node[match_function.end() + 1 :]
    
    return (label_node1, label_node2, int(bb))
    
    

In [106]:
(label_node1, label_node2, bb) = create_labels(node, analyse[1])
print(label_node1)
print(label_node2)
print(bb)

"{ FREQ:0 |\<bb\ 3\>:\l\
|call\ foo\ (b);\l\
}"
"{ FREQ:0 |\<bb\ 4\>:\l\|return\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 4\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
}"
3


#### Update the value of the nodes: name and labels

In [107]:
def update_node(node, attribute, value):
    """
        Update the node's attribute value.
        
        :param node: the node updated
        :param attribute: the attribute
        :param value: the value
    """
    node.set(attribute, value)
    return

In [108]:
label_before = node.get_attributes()["label"]
update_node(node, "label", "new_label of the node");
print(node.get_attributes()["label"])
update_node(node, "label", label_before)

new_label of the node


In [109]:
def update_bb_string(string, bb):
    """ 
        update recursivly the values of the bb found in the string. The first string is a node label
        
        :param string: the string
        :param bb: the bb
        :return: the new string
        :rtype: string
    """
    match = re.search(r'<bb\\\ \d+\\>', string)
    
    if match == None:
        return string
    match_bb = int(re.search(r'\d+', match.group()).group())
    
    if match_bb > bb:
        string = string.replace('<bb\ ' + str(match_bb), '<bb\ ' + str(match_bb + 1))
    return string[0:match.end()] + update_bb_string(string[match.end():], bb)

In [110]:
print(node.get_attributes()["label"])
print("----------")
print(update_bb_string(node.get_attributes()["label"], 0))

"{ FREQ:0 |\<bb\ 3\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 4\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
}"
----------
"{ FREQ:0 |\<bb\ 4\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 6\>;\ [0.00%]\l\
}"


In [111]:
def update_name(node_name, bb):
    current_number = int(re.search(r'\d+', re.search(r'block_\d+', node_name).group()).group())
    if bb < current_number:
        return node_name.replace('block_' + str(current_number), 'block_' + str(current_number+1))
    return node_name

In [112]:
print(node.get_name())
print("----------")
print(update_name(node.get_name(), 0))

fn_1_basic_block_3
----------
fn_1_basic_block_4


In [113]:
def update_nodes(nodes, bb):
    """
        Updates all the nodes whose bb are greater or equal to the bb given.
        
        :param nodes: the nodes to update
        :param bb: the bb
    """
    
    for node in nodes:
        node.set("label", update_bb_string(node.get_attributes()["label"], bb))
        node.set_name(update_name(node.get_name(), bb))


In [114]:
print(node.get_attributes()["label"] + "\n" + node.get_name())
print("----------")
update_nodes([node], bb)
print(node.get_attributes()["label"] + "\n" + node.get_name())

"{ FREQ:0 |\<bb\ 3\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 4\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
}"
fn_1_basic_block_3
----------
"{ FREQ:0 |\<bb\ 3\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 6\>;\ [0.00%]\l\
}"
fn_1_basic_block_3


#### On veut récupérer les noeuds qui pourront être update ce qui veut dire qu'il faut itérer dans les sous-graphes enfants et parents

In [115]:
def get_nodes_to_update(subgraph, graph_name, starter=0):
    """
        Retrive all the nodes to update from the parents to the children.
        
        :param subgraph: the current subgraph
        :param graph_name: the name of the global graph
        :param starter: the flag, 0 if you are the subgraph starter, 1 else
        :return nodes: the nodes to update
        :rtype: list
    """
    global subgraphs, parents
    
    nodes = extract_nodes(subgraph)
    
    for child_subgraph in subgraph.get_subgraph_list():
        nodes += get_nodes_to_update(child_subgraph, graph_name, 1)
    
    parent = parents[subgraph.get_name()].get_name()
    while(parent != graph_name and starter == 0):
        nodes += extract_nodes(parents[subgraph.get_name()])
        parent = parents[parent].get_name()
    return nodes

In [116]:
nodes_while = get_nodes_to_update(subgraphs["1_1"], G.get_name())
string_while = ""
for node in nodes_while:
    string_while += node.get_name() + " "
nodes_main = get_nodes_to_update(subgraphs["main"], G.get_name())
string_main=""
for node in nodes_while:
    string_main += node.get_name() + " "
print(string_while + "\n")
print(string_main)

fn_1_basic_block_3 fn_1_basic_block_4 fn_1_basic_block_5 fn_1_basic_block_6 fn_1_basic_block_0 fn_1_basic_block_1 fn_1_basic_block_2 fn_1_basic_block_7 fn_1_basic_block_8 

fn_1_basic_block_3 fn_1_basic_block_4 fn_1_basic_block_5 fn_1_basic_block_6 fn_1_basic_block_0 fn_1_basic_block_1 fn_1_basic_block_2 fn_1_basic_block_7 fn_1_basic_block_8 


#### Create the new node

In [117]:
def create_new_node(subgraph, prev_node, label, bb):
    return add_node(subgraph, update_name(prev_node.get_name(), bb-1), label=update_bb_string(label, bb))

In [118]:
new_node = create_new_node(subgraphs["main"], node, "oiseau", 8)
print(node.get_name())
print(new_node.get_name())
print(new_node.get_label())

fn_1_basic_block_8
fn_1_basic_block_9
oiseau


#### For all the subgraph, we need to update their names because there is a syntax error during the output otherwise

In [119]:
def recreate_subgraphs_name():
    global subgraphs
    for (name, subgraph) in subgraphs.items():
        subgraph.set_name("\"" + subgraph.get_name() + "\"")

In [120]:
recreate_subgraphs_name()
print(G.get_subgraph_list()[0].get_name())

"foo"


In [121]:
def main(file):
    global subgraphs, parents
    graph = graph_from_dot(file)
    subgraphs = {}
    parents = {}
    extract_subgraphs([graph])
    
    for (name, subgraph) in subgraphs.items():
        nodes = extract_nodes(subgraph)
        for node in nodes:
            (name_function, result) = analyse_label(node)
            if name_function is not None:
                (label_node1, label_node2, bb) = create_labels(node, result)
                node.set_label(label_node1)
                nodes_to_update = get_nodes_to_update(subgraph, graph.get_name())
                update_nodes(nodes_to_update, bb)
                create_new_node(subgraph, node, label_node2, bb)
    recreate_subgraphs_name()
    return graph

In [123]:
graph = main("../Tests/while_with_foo/test.c.011t.cfg.dot")

In [124]:
im_name = '{}.png'.format("/home/julien/Dev/Cassiopee-Control-Flow-Graph/Notebooks/outputfdd")
graph.write_png(im_name)